In [736]:
import numpy as np
import matplotlib.pyplot as plt
import json
import pandas as pd
from pandas import read_csv
from sklearn.utils import shuffle
from datetime import datetime

In [737]:
#lade einfache Daten und wandle in einen Dataframe um
with open('fullData.json') as dataJson:
    data = json.load(dataJson)

In [738]:
df_data = pd.DataFrame(data)
df = df_data[df_data['session'].notna()]

df.head()

,clientId,elementId,timeStamp,type,siteName,date,clientX,clientY,session,height,width,browser,key
339,2567c85a-ad16,inputPet,824.2999999988824,mouseover,TestEinfach,2022-10-17T13:35:24.171Z,673,363,1ff12350-5d06-5a07-9f0d-96a9a3942adb,944,1920,NaN,NaN
340,2567c85a-ad16,inputPet,858.2999999988824,mouseout,TestEinfach,2022-10-17T13:35:24.205Z,772,333,1ff12350-5d06-5a07-9f0d-96a9a3942adb,944,1920,NaN,NaN
341,2567c85a-ad16,inputCountry,2005.2999999988824,mouseover,TestEinfach,2022-10-17T13:35:25.361Z,796,266,1ff12350-5d06-5a07-9f0d-96a9a3942adb,944,1920,NaN,NaN
342,2567c85a-ad16,inputCountry,2296.5,focus,TestEinfach,2022-10-17T13:35:25.652Z,NaN,NaN,1ff12350-5d06-5a07-9f0d-96a9a3942adb,944,1920,Chrome,NaN
343,2567c85a-ad16,inputHouse,2875.2999999988824,mouseover,TestEinfach,2022-10-17T13:35:26.230Z,995,264,1ff12350-5d06-5a07-9f0d-96a9a3942adb,944,1920,NaN,NaN


In [739]:
#get all events which are actually in TestSchwer
array_input_schwer = ['inputHobby','inputCancelor','inputTemperature','inputDays','inputRiver', 'inputDifficultQuestion', 'clickHere', 'buttonYes', 'alibiClick']

df_einfach_without_schwerInput = df[~df['elementId'].isin(array_input_schwer)]
df_schwer_without_einfachInput = df[df['elementId'].isin(array_input_schwer)]

df_actually_schwer = df_einfach_without_schwerInput.loc[df_einfach_without_schwerInput['siteName'] == 'TestSchwer']
df_easy = df_actually_schwer.assign(sitename='TestSchwer')

df_einfach = df_einfach_without_schwerInput.loc[~(df_einfach_without_schwerInput['siteName'] == 'TestSchwer')]
df_schwer = pd.concat([df_easy, df_schwer_without_einfachInput])

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [740]:
#Gleich viele sessions bei TestSchwer und TestEinfach
sessionIdSchwer = df_schwer['session']
sessionIdEinfach = df_einfach['session']

df_einfach['is_duplicated'] = df_einfach.duplicated('session')
#86 ids auf Seite Einfach
setEinfach = set(sessionIdEinfach)
length = len(setEinfach)

#59 ids auf Seite Schwer
df_schwer['is_duplicated'] = df_schwer.duplicated('session')
setSchwer = set(sessionIdSchwer)
lengthSchwer = len(setSchwer)

in_schwer_but_not_in_einfach = setSchwer - setEinfach
in_einfach_but_not_in_schwer = setEinfach - setSchwer

array_einfach = list(in_einfach_but_not_in_schwer)
array_schwer = list(in_schwer_but_not_in_einfach)

#66 Teilnehmer bzw. Sessions?
df_einfach_without_ids = df_einfach[~df_einfach['session'].isin(array_einfach)]
df_schwer_without_ids = df_schwer[~df_schwer['session'].isin(array_schwer)]

df_einfach_final = df_einfach_without_ids.drop(['clientId', 'is_duplicated'], axis=1)
df_schwer_final = df_schwer_without_ids.drop(['clientId', 'is_duplicated'], axis=1)

print(len(array_einfach))
print(len(array_schwer))

23
4


<ipython-input-740-8a7767e40469>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_einfach['is_duplicated'] = df_einfach.duplicated('session')


In [741]:
df_first = pd.concat([df_einfach_final, df_schwer_final])
df_tmp = shuffle(df_first)

df_first_cleaned_data = df_tmp.loc[df["type"] !="scroll"]
df_first_cleaned_data.to_pickle('firstCleanedData.pkl')


In [742]:
#save dataframe to file 
df_einfach_final.to_pickle('einfach.pkl') 
df_schwer_final.to_pickle('schwer.pkl')


In [743]:
df1 = pd.read_pickle('einfach.pkl') 

In [744]:
df_einfach_2 = pd.read_pickle('einfach.pkl') 
df_schwer_2 = pd.read_pickle('schwer.pkl') 

df_einfach_2.loc[df["type"] !="scroll"]
df_schwer_2.loc[df["type"] !="scroll"]

df_data = pd.concat([df_einfach_2, df_schwer_2])
df = shuffle(df_data)


#remove scroll
df_data_without_scroll = df.loc[df["type"] !="scroll"]

In [746]:
# new sessionId 
import random
from random import choice
import collections

array_of_ids_einfach = df_einfach['session'].value_counts().index.tolist()
array_of_ids_schwer = df_schwer['session'].value_counts().index.tolist()
numbers_schwer = []

random_numbers_ids_einfach = random.sample(range(101,1000), len(array_of_ids_einfach))
for i in range(len(random_numbers_ids_einfach)):
    random_numbers_ids_schwer = random.choice([i for i in range(101, 1000) if i not in random_numbers_ids_einfach])
    numbers_schwer.append(random_numbers_ids_schwer)

numbers = random_numbers_ids_einfach + numbers_schwer

df_einfach['session'] = df_einfach['session'].replace(array_of_ids_einfach, random_numbers_ids_einfach)
df_schwer['session'] = df_schwer['session'].replace(array_of_ids_schwer, numbers_schwer)

print([item for item, count in collections.Counter(numbers).items() if count > 1])

<ipython-input-746-0a15b26dec93>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_einfach['session'] = df_einfach['session'].replace(array_of_ids_einfach, random_numbers_ids_einfach)


ValueError: Replacement lists must match in length. Expecting 70 got 89 

In [ ]:
df_replaced_together = pd.concat([df_einfach, df_schwer])
df_replaced_together['siteName'] = df_replaced_together['siteName'].replace('TestEinfach', 0)
df_replaced_together['siteName'] = df_replaced_together['siteName'].replace('TestSchwer', 1)

# Hier geht es weiter mit dem Dataframe full Cleaned


In [ ]:
df_data_without_scroll['timeStamp'] = df_data_without_scroll['timeStamp'].astype(float)
df_data_without_scroll['clientX'] = df_data_without_scroll['clientX'].astype(float)
df_data_without_scroll['clientY'] = df_data_without_scroll['clientX'].astype(float)
df_data_without_scroll['height'] = df_data_without_scroll['height'].astype(float)

In [ ]:
df_data_without_scroll["clientY"] = df_data_without_scroll["clientY"].fillna(df_data_without_scroll["clientY"].mean())
df_data_without_scroll["clientX"] = df_data_without_scroll["clientX"].fillna(df_data_without_scroll["clientX"].mean())

In [ ]:
#rund 8886 Events in unserem ganzen Datensatz 
#das sind dann um die 6000 Events in unserem Trainingssatz

In [ ]:
df_data_without_scroll.to_pickle('fullCleanedData.pkl') 